In [1]:
import pandas as pd
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))  # adds parent directory
from experiments_lib import prompt_ilec_data, set_context_window_size, get_context_window, set_base_instructions

%load_ext autoreload
%autoreload 2

# Actuarial Intern

The intern's manager has described the following task:

>We'd like to select an appropriate expected basis for each insurance plan type. We expect that the basis will vary by issue year, but are open to other ideas.  Which of the bases would provide the best fit, if we are looking at the actual-to-expected ratio on an amount basis? The following expected bases are available:

* Expected_Death_QX7580E_by_Amount
* Expected_Death_QX2001VBT_by_Amount
* Expected_Death_QX2008VBT_by_Amount
* Expected_Death_QX2008VBTLU_by_Amount
* Expected_Death_QX2015VBT_by_Amount
* Expected_Death_QX7580E_by_Policy
* Expected_Death_QX2001VBT_by_Policy
* Expected_Death_QX2008VBT_by_Policy
* Expected_Death_QX2008VBTLU_by_Policy
* Expected_Death_QX2015VBT_by_Policy
* ExpDeathQx2015VBTwMI_byPol
* ExpDeathQx2015VBTwMI_byAmt

This exercise will assume the intern has very little knowledge of the mechanics of life insurance (mortality improvement, preferred classes, etc.), and the final analysis produced by the intern will reflect that.


## Setup LLM params

These are used by the LLM to manage history, the context window includes the last N questions (from the user) and answers (from the agent / LLM)

In [2]:
set_context_window_size(10)

In [3]:
get_context_window()

deque([], maxlen=10)

## Base Instructions

The custom code wrapping the agent framework allows for a "base instruction" to be included with every request, to provide guidance on the overall objective.

For the actuarial intern persona, we'll borrow from the manager's instructions, and assume the intern is a knowledgeable LLM user, and can prompt accordingly.

In [4]:
base_instructions = "I'm an actuary trying to determine the best expected basis in the ILEC_DATA for each plan type using the actual-to-expected ratio."\
    " Use the claim amount as actuals, and the columns ending with by_Amount or byAmt for expected. Ignore columns beginning with Cen."

set_base_instructions(base_instructions)

## Data Exploration

We'll assume the intern has very little SQL knowledge, and will rely heavily on the model for data analysis and exploration

In [5]:
print(prompt_ilec_data("How many differerent insurance plan types are there, list them"))

Running request...
There are 7 insurance plan types:
- Other
- Perm
- Term
- UL
- ULSG
- VL
- VLSG


In [6]:
prompt = "Calculate the actual-to-expected ratio by amount (Actual Amount / Expected Amount) for each expected basis and plan type."\
    "Which basis looks like the best fit based on the A/E ratio?"

print(prompt_ilec_data(prompt))

Running request...
I pulled the A/E by amount for each plan using Death_Claim_Amount as actuals and all expected columns ending in by_Amount or byAmt (ignoring any columns starting with Cen). Here are the A/E ratios and the best-fitting basis (closest to 1.00) per plan:

- Other
  - 7580E: 0.403
  - 2001 VBT: 0.559
  - 2008 VBT: 0.740
  - 2008 VBT LU: 0.495
  - 2015 VBT: 0.868
  - 2015 VBT w/MI: 0.866
  - Best fit: 2015 VBT

- Perm
  - 7580E: 0.547
  - 2001 VBT: 0.682
  - 2008 VBT: 0.856
  - 2008 VBT LU: 0.707
  - 2015 VBT: 0.974
  - 2015 VBT w/MI: 0.969
  - Best fit: 2015 VBT

- Term
  - 7580E: 0.312
  - 2001 VBT: 0.476
  - 2008 VBT: 0.720
  - 2008 VBT LU: 0.431
  - 2015 VBT: 0.860
  - 2015 VBT w/MI: 0.856
  - Best fit: 2015 VBT

- UL
  - 7580E: 0.582
  - 2001 VBT: 0.714
  - 2008 VBT: 0.878
  - 2008 VBT LU: 0.720
  - 2015 VBT: 1.014
  - 2015 VBT w/MI: 1.008
  - Best fit: 2015 VBT w/MI

- ULSG
  - 7580E: 0.518
  - 2001 VBT: 0.623
  - 2008 VBT: 0.733
  - 2008 VBT LU: 0.571
  - 2015 VBT:

### Actuarial Intern consults w/ Manager

Provides the following summary of the results to the manager. The intern isn't familiar with mortality improvement, and doesn't understand why mortality improvement should improve the A/E ratio, nor do they as for clarification.

> The 2015VBT is the best fit by amount for all plan types except UL and VL, which has is the closest 2015VBT w/MI. The 2015VBT and 2015VBT w/ MI are generally pretty close.

This finding surprises the manager, who expected that the 2015VBT w/ MI would be closer. She responds that:

> Can you find which attained ages, preferred classes and calendar years are a better fit w/ the MI basis vs. non-MI basis, by insurance plan type? Also, provide a final recommendation for the expected basis to use with each plan type.

In [7]:
prompt = "Identify which sections of the ILEC_DATA are fit better by the 2015VBT than the 2015VBT w/ MI, and vice versa/"\
    "Look at attained ages, preferred classes, and calendar years, either in combination or one at a time."

print(prompt_ilec_data(prompt, max_turns=500))

Running request...
Approach used
- Actuals: Death_Claim_Amount
- Expected: 2015 VBT by amount = Expected_Death_QX2015VBT_by_Amount; 2015 VBT w/ MI by amount = ExpDeathQx2015VBTwMI_byAmt
- Ignored any Cen* columns
- Compared A/E (sum actual / sum expected) and flagged the basis closer to 1.00 as “best”
- Reviewed by plan and: attained age bands (<40, 40–49, 50–59, 60–69, 70–79, 80+), preferred class (1–4 and null), and calendar year

Summary: where 2015 VBT vs 2015 VBT w/ MI fits better

By attained age band
- Other
  - 2015 VBT better: <40, 40–49, 50–59, 60–69, 80+
  - 2015 VBT w/ MI better: 70–79
- Perm
  - 2015 VBT better: 50–59, 60–69, 70–79, 80+
  - 2015 VBT w/ MI better: <40, 40–49
- Term
  - 2015 VBT better: <40, 40–49, 50–59, 60–69, 70–79
  - 2015 VBT w/ MI better: 80+
- UL
  - 2015 VBT better: 80+
  - 2015 VBT w/ MI better: <40, 40–49, 50–59, 60–69, 70–79
- ULSG
  - 2015 VBT better: all bands
  - 2015 VBT w/ MI better: none by age band
- VL
  - 2015 VBT better: 40–49, 50–59
  -

### Actuarial Intern consults w/ Manager

The intern provides the following findings to the manager:

> Calendar years 2016+ are generally closer to 2015VBT w/ MI. There is no consistent pattern with preferred classes, or attained ages.

The manager still isn't quite sure what to think, but remembers that the AI platform can run a decision tree, which might shed some light on the situation by considering the three factors (attained age, calendar year, and preferred class) simultaneously to find the most credible explanation (in the actuarial sense of credibility.)  She suggests that the intern prompt the AI model to run a decision tree.

In [8]:
print(prompt_ilec_data("Run a decision tree for each plan type using the outlined variables and provide a summary of your findings. Offer possibly theories about why the MI basis doesn't fit well.", max_turns=500))

Running request...
Here’s what I ran and what it shows.

How I set this up
- Actuals: Death_Claim_Amount
- Expected candidates (by amount): Expected_Death_QX7580E_by_Amount, Expected_Death_QX2001VBT_by_Amount, Expected_Death_QX2008VBT_by_Amount, Expected_Death_QX2008VBTLU_by_Amount, Expected_Death_QX2015VBT_by_Amount, ExpDeathQx2015VBTwMI_byAmt (ignored any columns beginning with Cen)
- Decision-tree x-variables (as requested): Attained_Age, Preferred_Class, Observation_Year
- For each plan, I first identified the best overall basis (A/E closest to 1.00) and then ran a Poisson decision tree (max depth 3, cp ~0.01) with that basis as the offset. The tree’s node y-values are A/E (actual-to-expected).

Best expected basis by plan (overall A/E)
- Other: 2015 VBT, A/E = 0.8685 (MI = 0.8656; 2015 VBT is very slightly closer to 1)
- Perm: 2015 VBT, A/E = 0.9744 (MI = 0.9693)
- Term: 2015 VBT, A/E = 0.8603 (MI = 0.8556)
- UL: 2015 VBT w/ MI, A/E = 1.0080 (2015 VBT = 1.0137)
- ULSG: 2015 VBT w/

### Final Analysis

The intern provides the following final analysis to their manager using the above. They find that synthesizing the above into a cohorent message is a substantial amount of work.

I’ve completed my analysis and recommendations:
* Use 2015 VBT w/ MI for UL, VL, ULSG, VLSG 
* Use 2015 VBT for Other, Perm, Term
* In general, the best preferred class has A/E < 100%, and worse preferred classes have A/E > 100%.

Here are some additional details by plan type:

**Plan Type: “Other”**
* 2015VBT produces the closest A/E ratio by amount, at 86.85%.  2015VBT w/ MI is at 86.56% (very close).
  * Calendar years 2014.5+ A/Es are substantially lower than 2015VBT at 46%
  * In calendar years <2014.5, preferred classes 2 and 3 fit well, but 1 does not (high A/E)

**Plan Type: “Perm”**
* 2015 VBT A/E is the best at 97.44%.  2015VBT w/ MI is close second at 96.93%.
  * Preferred class #1 has the lowest A/E @ 71% for ages < 85.5, but increases to 93% for older ages.
  * Preferred classes 2-4 are close to 100%.
  * Younger ages (<51.5) are high at 112% A/E

**Plan Type: “Term”**
* 2015VBT has the best A/E at 86.03%, and the 2015VBT w/ MI is at 85.56%
  * Best classes and middle ages are too conservative (A/E < 100%)
  * Preferred class #1 is low overall @ 69%. Classes 3-4 are near or above 100%
  * Very old / young ages are at or over 100%

**Plan Type: “UL”**
* 2015VBT w/ MI has the best A/E at 100.01%, and the 2015VBT is at 101.37%
  * Ages 87.5+ are about 84% A/E, and 
  * For ages < 87.5:
  * Preferred class #1 is low at 86%
  * Classes 2-4 are high at about 115%

**Plan Type: “ULSG”**
* 2015VBT w/ MI has the best A/E at 85.21%, and 2015VBT is a close second at 85.4%. 
  * Classes #1 and #2 are low, ranging from 71% - 85%
  * Classes #3 and #4 look OK at 96%

**Plan Type: “VL”**
* 2015VBT w/ MI has the best A/E at 99.99%, and the 2015VBT is at 100.70%
  * Preferred class #1 is at 83%
  * Preferred classes 2-4 are about 106%

**Plan Type: “VLSG”**
* 2015VBT w/ MI has the best A/E at 97.21%, and the 2015VBT is at 97.42%
  * Overall fits well, nothing material to call-out




### Additional Notes

Logical fallacy w/ in response

> ULSG: 2015 VBT w/ MI, A/E = 0.8521 (2015 VBT = 0.8540; essentially a tie, MI slightly closer)

> (...)

> VLSG: 2015 VBT w/ MI, A/E = 0.9721 (2015 VBT = 0.9742; essentially a tie, MI slightly closer)